In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import axes
import pickle
from datetime import datetime
from scipy import log,exp,sqrt,stats
print('Packages')

Packages


In [ ]:
# Importing pickle file
data1 = np.load('/Users/louisribieras/Desktop/Invictus/Data/latest_coin_data.pkl')
data2 = np.load('/Users/louisribieras/Desktop/Invictus/Data/additional_data.pkl')

# Removing July dates from data1
data1 = data1.drop(data1.columns[0:4], axis = 1)

# Combining the two dataframes
data = pd.concat([data2, data1], axis = 1, join = 'outer', sort = True)

# Filling Nan values with 0
data = data.dropna()

# Saving original date index
date_index = data.columns

print('Loading of Data')
print('shape of data: ', data.shape)

In [ ]:
data.head(3)

In [ ]:
print('Getting Market Caps')
# Initialize a close_price list
mc_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        mc_list.append(data[column][date]['market_cap'])

In [ ]:
print('Making Market Cap dataframe')
# Splitting list into 133 lists (one per coin)
mc_list = [mc_list[i:i+947] for i  in range(0, len(mc_list), 947)]   
# Making dataframe
mc_data = pd.DataFrame(data = mc_list, index = data.index, columns = data.columns)

In [ ]:
mc_data.head(3)

In [ ]:
print('Getting Close Prices')
# Initialize a close_price list
cp_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        cp_list.append(data[column][date]['close_price'])

In [ ]:
print('Making Close Price dataframe')
# Splitting list into 133 lists (one per coin)
cp_list = [cp_list[i:i+947] for i  in range(0, len(cp_list), 947)]   
# Making dataframe
close_data = pd.DataFrame(data = cp_list, index = data.index, columns = data.columns)

In [ ]:
close_data.head(3)

In [ ]:
print('flipping the dates in order')
# Reording the date index
close_data = close_data[close_data.columns[::-1]]
mc_data = mc_data[mc_data.columns[::-1]]
data = data[data.columns[::-1]]

In [ ]:
print('Converting date index to datetime index')
# Index to Datetime index
mc_data.columns = pd.to_datetime(mc_data.columns)
close_data.columns = pd.to_datetime(close_data.columns)

In [ ]:
print('Share cap functions')

def share_cap_recurse(market_cap_list, num_coins, cap_ceiling_amount, final_cap_list):

    not_realloc_val = 0

    realloc_val = 0

    num_final = 0

    new_market_cap_list = []

    

    for i in range(0, len(market_cap_list)):

        if(market_cap_list[i] > cap_ceiling_amount):

            final_cap_list.append(cap_ceiling_amount)

            realloc_val += market_cap_list[i] - cap_ceiling_amount

            num_final += 1

        else:

            not_realloc_val += market_cap_list[i]

     

    if(num_final!=0):

        new_market_cap_list = [x * (1+(realloc_val/not_realloc_val)) for x in market_cap_list[num_final:]]

    else:

        new_market_cap_list = deepcopy(market_cap_list)

    

    if len(final_cap_list) == num_coins:

        return

    elif (new_market_cap_list[0] <= cap_ceiling_amount): #assumes ordered desc in market cap

        for cap in new_market_cap_list:

            final_cap_list.append(cap)

        return       

    else:

        share_cap_recurse(new_market_cap_list, num_coins, cap_ceiling_amount, final_cap_list)

        

def share_cap(market_cap_list, cap_ceiling_percent):  #CHANGE CAP CEILING HERE

    total_cap = sum(market_cap_list)

    cap_ceiling_amount = total_cap * cap_ceiling_percent

    final_cap_list = []

    

    share_cap_recurse(market_cap_list, len(market_cap_list), cap_ceiling_amount, final_cap_list)

    

    return (final_cap_list)

In [ ]:
print('Reverting Index code')
# Reverting Index code
# mc_data.columns = date_index
# close_data.columns = date_index

In [ ]:
# Selecting dates and median market caps
selectedmarket = mc_data.copy()
selectedmarket = selectedmarket.loc[:, 'Jul 01, 2017':'Jul 01, 2018']
selectedmarket = selectedmarket.resample('W',axis=1).median()

# Selecting dates and median close price
selectedclose = close_data.copy()
selectedclose = selectedclose.loc[:, 'Jul 01, 2017':'Jul 01, 2018']
selectedclose = selectedclose.resample('W',axis=1).median()

# Calculating percent change for returns
selectedclose1 = selectedclose.copy().T
selectedreturns = selectedclose1.pct_change(1).T
selectedreturns.head()
print('selected market, selectedclose and selectedreturns')

In [ ]:
print('Total market cap movement for tethershare')
tmc_list = list()

for column in close_data.columns:
    tmc_list.append(close_data[column].sum())
    
tmc_data = pd.DataFrame(data = tmc_list, columns = ['total_mc'])
tmc_data.index = close_data.columns
tmc_pct = tmc_data.pct_change(1)

# Resampling the tmc data to get weekly
tmc_pct = tmc_pct.loc['Jul 01, 2017':'Jul 01, 2018',:]
tmc_pct = tmc_pct.resample('W').median()
tmc_pct_index = tmc_pct.index

tmc_pct.head(5)

In [ ]:
num_index = np.arange(53)
tmc_pct.index = num_index

usdt_share = {}
usdt_share[0] = 0.10      #USDT starts off with ___% for the date


for i in tmc_pct.index[1:]:
    if tmc_pct['total_mc'][i] < -0.01:
        usdt_share[i] = usdt_share[i -1] - 3*(tmc_pct['total_mc'][i])
        if usdt_share[i] >= 0.5:
            usdt_share[i] = 0.5
    elif tmc_pct['total_mc'][i] > 0.01: 
        usdt_share[i] = usdt_share[i -1] - 3*(tmc_pct['total_mc'][i])
        if usdt_share[i] <= 0:
            usdt_share[i] = 0
    else:
        usdt_share[i] = usdt_share[i -1]

In [ ]:
ilist = range(0, len(tmc_pct))

tethershare = {}
         
for (i, column) in zip(ilist, selectedmarket):
    tethershare[column] = usdt_share[i]
         
print('setting tethershare labels to match dates')

In [ ]:
# Calculating the returns on the tether held
usdt_close = pd.DataFrame(data = selectedclose.loc['tether'])
usdt_returns = usdt_close.pct_change(1)

In [ ]:
print('loop for C10 with tether market cap function')
#FOR LOOP FOR C10 with tether
topmc10t = {}
sc10t = {}
tc10t = {}
ps10t = {}
invest10t = {}
returns10t = {}
lg_returns10t = {}
selectedmc10t = {}
topcoins10t = {}
c10sharet = {}
tetherreturns = {}

#for every date
for column in selectedmarket:
    #sort in descending order
    selectedmc10t[column] = selectedmarket[column].sort_values(ascending=False)
    
    #take the top coins
    topmc10t[column] = selectedmc10t[column].head(10)
    #take the index of the top coins
    topcoins10t[column] = topmc10t[column].index
    
    #get the share cap of the top 10
    sc10t[column] = share_cap(topmc10t[column], 0.15)      #CHOICE OF CAP CEILING
    
    #total cap of top 10
    tc10t[column] = sum(sc10t[column])
    
    #percentage of total fund allocated to tether/C10- NEW CODE!!
    c10sharet[column] = 1 - tethershare[column]
    
    #percentage share to be allocated for top 10 - NEW CODE!!!
    ps10t[column] = [x * c10sharet[column] / tc10t[column] for x in sc10t[column] ]
    
    #building dataframe of top20 coins by mc and %share to be invested
    invest10t[column] = pd.DataFrame()
    invest10t[column] = pd.DataFrame({'coin': topcoins10t[column], 'pct_share': ps10t[column]})
    invest10t[column] = invest10t[column].set_index('coin')
        
        
    #building dataframe of top 20 coins by mc and close price of that week
    for name in selectedreturns:
        returns10t[column] = pd.DataFrame()
        returns10t[column] = pd.DataFrame({'returns': selectedreturns[column]})
        returns10t[column] = returns10t[column].loc[topcoins10t[column]]
        
     #for loop to generate portfolio returns for a particular week
    portfolio_simple_returns10t = {}
    portfolio_total_returns10t = list()
   
    #tether returns
    tetherreturns[column] = tethershare[column]* usdt_returns.loc[column, 'tether']

    for i, df in invest10t.items():
        portfolio_simple_returns10t[i] = (invest10t[i].transpose().dot(returns10t[i])) + tetherreturns[i]
        portfolio_total_returns10t.append(portfolio_simple_returns10t[i]['returns'])      

In [ ]:
print('cleaning tether results')
#FOR C10 compiling and cleaning the table into a readable format WITH TETHER
simple_returns10t = pd.DataFrame(data = portfolio_total_returns10t) 
simple_returns10t.columns = ['weekly_portfolio_simple_returns']
simple_returns10t['date'] = selectedreturns.T.index
simple_returns10t = simple_returns10t.set_index('date')

In [ ]:
#FOR C10 with tether
final_cum10t = (1 + simple_returns10t).cumprod()
final_cum10t.index = pd.to_datetime(final_cum10t.index)

In [ ]:
# formatting the final dataframes and then csv file (tether)
final_tether = simple_returns10t.merge(final_cum10t, how='outer', left_index=True, right_index=True)
final_tether.columns = ['weekly_simple_returns', 'cumulative_returns']
final_tether.at['2017-07-02', 'cumulative_returns'] = 1

In [ ]:
#FOR LOOP FOR C10
topmc10 = {}
sc10 = {}
tc10 = {}
ps10 = {}
invest10 = {}
returns10 = {}
lg_returns10 = {}
selectedmc10 = {}
topcoins10 = {}
c10share = {}
tetherreturns = {}

#for every date
for column in selectedmarket:
    #sort in descending order
    selectedmc10[column] = selectedmarket[column].sort_values(ascending=False)
    
    #take the top coins
    topmc10[column] = selectedmc10[column].head(10)
    #take the index of the top coins
    topcoins10[column] = topmc10[column].index
    
    #get the share cap of the top 10
    sc10[column] = share_cap(topmc10[column], 0.15)      #CHOICE OF CAP CEILING
    
    #total cap of top 10
    tc10[column] = sum(sc10[column])
    
    #percentage of total fund allocated to tether/C10- NEW CODE!!
    #c10share[column] = 1 - tethershare[column]
    
    #percentage share to be allocated for top 20 - NEW CODE!!!
    ps10[column] = [x  / tc10[column] for x in sc10[column] ]
    
    #building dataframe of top20 coins by mc and %share to be invested
    invest10[column] = pd.DataFrame()
    invest10[column] = pd.DataFrame({'coin': topcoins10[column], 'pct_share': ps10[column]})
    invest10[column] = invest10[column].set_index('coin')
        
        
    #building dataframe of top 20 coins by mc and close price of that week
    for name in selectedreturns:
        returns10[column] = pd.DataFrame()
        returns10[column] = pd.DataFrame({'returns': selectedreturns[column]})
        returns10[column] = returns10[column].loc[topcoins10[column]]
        
     #for loop to generate portfolio returns for a particular week
    portfolio_simple_returns10 = {}
    portfolio_total_returns10 = list()

    for i, df in invest10.items():
        portfolio_simple_returns10[i] = (invest10[i].transpose().dot(returns10[i]))
        portfolio_total_returns10.append(portfolio_simple_returns10[i]['returns'])  

In [ ]:
print('cleaning tether results')
#FOR C10 compiling and cleaning the table into a readable format WITHOUT TETHER
simple_returns10 = pd.DataFrame(data = portfolio_total_returns10) 
simple_returns10.columns = ['weekly_portfolio_simple_returns']
simple_returns10['date'] = selectedreturns.T.index
simple_returns10 = simple_returns10.set_index('date')

In [ ]:
#FOR C10
final_cum10 = (1 + simple_returns10).cumprod()
final_cum10.index = pd.to_datetime(final_cum10.index)

In [ ]:
# formatting the final dataframes and then csv file (C10)
final_c10 = simple_returns10.merge(final_cum10, how='outer', left_index=True, right_index=True)
final_c10.columns = ['weekly_simple_returns', 'cumulative_returns']
final_c10.at['2017-07-02', 'cumulative_returns'] = 1

In [ ]:
final_tether.head(5)

In [ ]:
final_c10.head(5)

In [ ]:
plt.ylabel('Percentage Change /100%')
plt.rcParams['figure.figsize'] = (30,15)
plt.plot(final_c10, label="C10")
plt.plot(final_tether['cumulative_returns'], label="C10 with tether (mc)")
plt.title('C10 vs C10 w/Tether (market cap)')
plt.legend()
plt.show()

In [ ]:
final_tether.to_csv("C10 tether 3x.csv")